In [5]:
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, SGD

# Define the function to create the model architecture
def get_model(num_users, num_items, layers, reg_layers):
    assert len(layers) == len(reg_layers), "Each layer should have a corresponding regularization parameter"
    
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    item_input = Input(shape=(1,), dtype='int32', name='item_input')

    # Using TensorFlow Keras initializers and regularizers
    embedding_initializer = RandomNormal(mean=0.0, stddev=0.01)
    user_embedding = Embedding(input_dim=num_users, output_dim=layers[0]//2,
                               embeddings_initializer=embedding_initializer,
                               embeddings_regularizer=l2(reg_layers[0]),
                               name='user_embedding')(user_input)
    item_embedding = Embedding(input_dim=num_items, output_dim=layers[0]//2,
                               embeddings_initializer=embedding_initializer,
                               embeddings_regularizer=l2(reg_layers[0]),
                               name='item_embedding')(item_input)

    # Flatten the embedding output to remove the sequence dimension
    user_latent = Flatten()(user_embedding)
    item_latent = Flatten()(item_embedding)

    # Concatenate the user and item embeddings
    vector = Concatenate()([user_latent, item_latent])

    # Add MLP layers as specified in the layers list
    for idx, layer_size in enumerate(layers[1:], start=1):  # Start from 1 since layer[0] is for embeddings
        vector = Dense(layer_size, activation='relu',
                       kernel_regularizer=l2(reg_layers[idx]),
                       name=f'layer{idx}')(vector)

    # Prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name='prediction')(vector)

    model = Model(inputs=[user_input, item_input], outputs=prediction)
    return model

# Function to generate predictions
def generate_predictions(model, num_users, num_items):
    user_input = np.repeat(np.arange(num_users), num_items)
    item_input = np.tile(np.arange(num_items), num_users)
    predictions = model.predict([user_input, item_input], batch_size=256, verbose=1)
    return user_input, item_input, predictions

# Define parameters
num_users = 6040  # Example: set the actual number of users from your dataset
num_items = 3706  # Example: set the actual number of items from your dataset
layers = [64, 32, 16, 8]  # Example: set the layers as per your trained model
reg_layers = [0, 0, 0, 0]  # Example: set the regularization for each layer

# Initialize and compile the model
model = get_model(num_users, num_items, layers, reg_layers)
learner = 'adam'  # Example: set the optimizer used during training
learning_rate = 0.001  # Example: set the learning rate used during training

if learner.lower() == "adagrad": 
    model.compile(optimizer=Adagrad(learning_rate=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(learning_rate=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(learning_rate=learning_rate), loss='binary_crossentropy')

# Load the trained weights
model_out_file = 'C:/Users/Sten Stokroos/Desktop/Thesis2.0/zelf/neural_collaborative_filtering/Pretrain/ml-1m_MLP_64_32_16_8_1717442468.weights.h5'  # Change TIMESTAMP to the actual timestamp
model.load_weights(model_out_file)

# Generate predictions
user_input, item_input, predictions = generate_predictions(model, num_users, num_items)
prediction_matrix = np.zeros((num_users, num_items))
for user, item, prediction in zip(user_input, item_input, predictions):
    prediction_matrix[user, item] = prediction[0]  # Extract scalar from array

# Save predictions to CSV
print(prediction_matrix.shape)
import pandas as pd
pred_df = pd.DataFrame(prediction_matrix)
print(pred_df.shape)

c:\Users\Public\ana\envs\new\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


87439/87439 ━━━━━━━━━━━━━━━━━━━━ 111s 1ms/step
(6040, 3706)
(6040, 3706)


In [6]:
pred_out_file = 'C:/Users/Sten Stokroos/Desktop/Thesis2.0/zelf/neural_collaborative_filtering/Data/predicted_scores_copy.csv'
pred_df.to_csv(pred_out_file, index=False)

In [7]:
df = pd.read_csv('C:/Users/Sten Stokroos/Desktop/Thesis2.0/zelf/neural_collaborative_filtering/Data/predicted_scores_copy.csv')
confounder_data = df.to_numpy()
# Transpose the array if needed
confounder_data = confounder_data.T
print(confounder_data.shape)

(3706, 6040)
